<p align="center">
<img src="Images/sorbonne_logo.png" alt="Logo" width="300"/>
</p>

# **PANEL DATA ECONOMETRICS - Code File**

* **Author**: Elia Landini
* **Student ID**: 12310239
* **Course**: EESM2-Financial Economics 
* **Class**: Panel Data Econometrics
* **Supervisor**: Jean-Bernard Chatelain 
* **Reference Paper**: Lofaro, A., & Di Bucchianico, S. (2025). Impact of monetary policy on functional income distribution: A panel vector autoregressive analysis. Economic Modelling, 107227
* **Reference Repository**: https://github.com/EliaLand/PVAR_MonetaryPolicy_FunctionalIncome

### **0) REQUIREMENTS SET-UP**

In [55]:
# Requirements.txt file installation
# !pip install -r requirements.txt

In [56]:
# Libraries import
import warnings
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import levene
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import sklearn.tree
import sklearn.metrics
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing 
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix,
                             precision_score, recall_score, f1_score,
                             accuracy_score, precision_recall_curve, auc, 
                             RocCurveDisplay, ConfusionMatrixDisplay)
from sklearn.linear_model import (LinearRegression, LogisticRegression)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight
import plotly.express as px
import openpyxl as pxl
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
from IPython.display import Image
import itertools
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from ann_visualizer.visualize import ann_viz
from collections import Counter
import shap

In [57]:
# Statistical Significance labelling 
def significance_stars(p):
    if p < 0.001:
        return "***"  
    elif p < 0.01:
        return "**"    
    elif p < 0.05:
        return "*"   
    else:
        return ""

In [58]:
# We supress potential warnings with this command
warnings.filterwarnings("ignore")

### **1) PART 1 - DATASET, UNIVARIATE & BIVARIATE DESCRIPTIVE STATISTICS AFTER TRANSFORMATIONS**

##### <span style="color: dodgerblue"> **1.1) In your data set, which are the variables which are varying with respect to two indices (or more) if you consider inflows and outflows from one individual or country to another individual or countries? Which are the variables which are varying only with respect to time? Which are the variables which are varying only with respect to individuals?** </span>

In [59]:
# Data loading 
# Data source: https://data.mendeley.com/datasets/dbfjwwwzgf/1 

raw_data = pd.read_excel("Data/Dataset_MP_Impact_functional_Distribution.xlsx")

# Transfor year in datetime format 
# !!! With this we avoid to include the year column as a numeric column in the descriptive statistics computations
raw_data["year"] = pd.to_datetime(raw_data["year"], format="%Y")

raw_data

,year,country,i,P,W,WR,GDP,LS,PCOM,UN,SHORTUN,LONGUN,LF,REER,SH
0,1970-01-01,Australia,7.108333,9.1,6.413648,76.120158,2.835193e+05,59.7,2.113115,1.653364,NaN,NaN,5478.225,93.904724,7.108333
1,1971-01-01,Australia,7.008333,9.6,6.909308,77.071751,2.946057e+05,60.4,2.709369,1.907799,NaN,NaN,5622.972,97.030332,7.008333
2,1972-01-01,Australia,5.083333,10.2,7.284607,75.808223,3.023225e+05,60.3,2.925285,2.610752,NaN,NaN,5750.258,98.243273,5.083333
3,1973-01-01,Australia,6.983333,11.1,8.096658,74.465333,3.147389e+05,60.8,4.174010,2.309362,NaN,NaN,5899.899,109.985873,6.983333
4,1974-01-01,Australia,13.158333,12.9,9.688749,75.817692,3.189447e+05,66.9,13.802962,2.670259,NaN,NaN,6052.784,111.086199,13.158333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2015-01-01,USA,0.226667,100.0,65.600000,62.655205,1.820602e+07,56.8,66.099106,5.279920,5968.333,2328.0,157129.900,115.215974,-1.142704
746,2016-01-01,USA,0.644167,101.3,66.400000,62.819300,1.850960e+07,56.8,56.023884,4.869110,5746.000,2005.0,159187.200,118.668129,0.462412
747,2017-01-01,USA,1.152500,103.4,68.600000,63.695450,1.892457e+07,56.9,69.357989,4.355201,5294.250,1688.0,160319.800,118.325019,0.956409
748,2018-01-01,USA,2.188333,105.9,70.900000,64.279238,1.948198e+07,56.8,89.358138,3.895676,4964.917,1349.0,162075.000,117.086340,1.940361


In [60]:
# Descriptive data
# Total dataset statistics
    
raw_data.describe()

,year,i,P,W,WR,GDP,LS,PCOM,UN,SHORTUN,LONGUN,LF,REER,SH
count,750,750.000000,750.000000,740.000000,740.000000,7.500000e+02,740.000000,750.000000,741.000000,588.000000,588.000000,741.000000,750.000000,750.000000
mean,1994-07-02 12:00:00,6.186977,64.343733,29.663542,49.244036,1.941396e+06,58.345405,47.449644,7.090870,1093.540056,822.288778,25015.810665,97.542743,5.709472
min,1970-01-01 00:00:00,-0.695000,1.700000,3.045484,13.615376,7.833901e+04,43.200000,2.113115,0.554685,36.500000,10.600000,1509.000000,64.192610,-6.956319
25%,1982-01-01 00:00:00,2.185590,40.550000,16.499738,38.634013,3.137606e+05,55.200000,22.325814,4.557832,163.868042,187.013999,4610.000000,90.770763,2.105984
50%,1994-07-02 12:00:00,5.315000,69.250000,28.064481,45.779160,8.989052e+05,58.300000,35.596160,6.776669,696.987875,396.700000,13526.010000,97.000461,5.267943
75%,2007-01-01 00:00:00,9.508333,90.475000,42.012400,54.778060,2.221882e+06,61.600000,69.357989,8.736210,1063.156250,1333.500000,27708.250000,101.753458,9.508333
max,2019-01-01 00:00:00,24.900000,110.800000,75.107968,193.613088,1.992897e+07,88.400000,135.215071,26.093716,9770.580000,6416.000000,163538.700000,160.023520,24.900000
std,NaN,4.846420,29.535459,16.237418,19.889039,3.088414e+06,5.415204,36.721151,3.961598,1685.300608,881.931090,32929.409955,12.833124,5.545043


In [61]:
# Descriptive data
# Single country statistics 
    
for country, df in raw_data.groupby("country"):
# We are interest only in numeric data 
    df = df.select_dtypes(include="number")
    print(f"\n=== {country} ===")
    print(df.describe())


=== Australia ===
               i           P          W         WR           GDP         LS  \
count  50.000000   50.000000  50.000000  50.000000  5.000000e+01  50.000000   
mean    7.739117   57.008000  29.756793  59.209667  6.611638e+05  58.062000   
std     4.416389   30.353656  15.048319   7.710353  2.934876e+05   4.013238   
min     1.344167    9.100000   6.413648  47.007258  2.835193e+05  52.300000   
25%     4.896667   30.550000  18.276142  54.178082  3.937023e+05  54.600000   
50%     6.566667   59.050000  28.264109  57.581440  5.860922e+05  57.850000   
75%    10.222500   81.225000  42.132904  62.012984  9.116328e+05  60.475000   
max    17.611667  106.900000  55.305034  77.071751  1.209289e+06  66.900000   

             PCOM         UN     SHORTUN      LONGUN           LF        REER  \
count   50.000000  50.000000   42.000000   42.000000     50.00000   50.000000   
mean    47.449644   6.250230  380.560828  253.481952   9016.64966   95.102589   
std     37.069227   2.1273

##### <span style="color: dodgerblue"> **1.2) What is the largest number of period T for individuals? What is the number of individuals (countries)?** </span>

[no code, comment in the document]

##### <span style="color: dodgerblue"> **1.3) Comment on the structure of the unbalanced panel (how many (and which) countries have a single observation, discontinuities between observations, how many individuals have at least 2 consecutive observations (which is useful to compute lags, autocorrelations, first difference and within estimators)?** </span>

[no code, comment in the document]

##### <span style="color: dodgerblue"> **1.4) VARIABLE TRANSFORMATIONS PART 1: Compute between transformed and one-way-within transformed variables for all variables. Present a table with the the variance of the one-way-within-fixed-effects, between and pooled data for each variable. Compute the share of between and within variance in the total variance for each variable. Comment these results.** </span>

In [62]:
# Between Transformation 
# Key: the between transformation entails the computation of the overtime mean at the individual level for each numeric column. With this, we remove the time-variant dynamics to isolate the between-individuals variability (i). 

trans_data = raw_data.copy()

# !!! we are only interested in numeric cols
# !!! the command "columns" takes only the columns name and not the whole column (do not forget!)
numeric_cols = raw_data.select_dtypes(include="number").columns

trans_data[[f"{c}_between" for c in numeric_cols]] = (
    raw_data
    .groupby("country")[numeric_cols]
    .transform("mean")
)

trans_data

,year,country,i,P,W,WR,GDP,LS,PCOM,UN,...,WR_between,GDP_between,LS_between,PCOM_between,UN_between,SHORTUN_between,LONGUN_between,LF_between,REER_between,SH_between
0,1970-01-01,Australia,7.108333,9.1,6.413648,76.120158,2.835193e+05,59.7,2.113115,1.653364,...,59.209667,6.611638e+05,58.062,47.449644,6.250230,380.560828,253.481953,9016.64966,95.102589,7.739117
1,1971-01-01,Australia,7.008333,9.6,6.909308,77.071751,2.946057e+05,60.4,2.709369,1.907799,...,59.209667,6.611638e+05,58.062,47.449644,6.250230,380.560828,253.481953,9016.64966,95.102589,7.739117
2,1972-01-01,Australia,5.083333,10.2,7.284607,75.808223,3.023225e+05,60.3,2.925285,2.610752,...,59.209667,6.611638e+05,58.062,47.449644,6.250230,380.560828,253.481953,9016.64966,95.102589,7.739117
3,1973-01-01,Australia,6.983333,11.1,8.096658,74.465333,3.147389e+05,60.8,4.174010,2.309362,...,59.209667,6.611638e+05,58.062,47.449644,6.250230,380.560828,253.481953,9016.64966,95.102589,7.739117
4,1974-01-01,Australia,13.158333,12.9,9.688749,75.817692,3.189447e+05,66.9,13.802962,2.670259,...,59.209667,6.611638e+05,58.062,47.449644,6.250230,380.560828,253.481953,9016.64966,95.102589,7.739117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2015-01-01,USA,0.226667,100.0,65.600000,62.655205,1.820602e+07,56.8,66.099106,5.279920,...,48.940835,1.171694e+07,59.682,47.449644,6.202492,6285.456600,1655.700000,129297.66500,104.919930,4.988276
746,2016-01-01,USA,0.644167,101.3,66.400000,62.819300,1.850960e+07,56.8,56.023884,4.869110,...,48.940835,1.171694e+07,59.682,47.449644,6.202492,6285.456600,1655.700000,129297.66500,104.919930,4.988276
747,2017-01-01,USA,1.152500,103.4,68.600000,63.695450,1.892457e+07,56.9,69.357989,4.355201,...,48.940835,1.171694e+07,59.682,47.449644,6.202492,6285.456600,1655.700000,129297.66500,104.919930,4.988276
748,2018-01-01,USA,2.188333,105.9,70.900000,64.279238,1.948198e+07,56.8,89.358138,3.895676,...,48.940835,1.171694e+07,59.682,47.449644,6.202492,6285.456600,1655.700000,129297.66500,104.919930,4.988276


In [ ]:
# Within Transformation 
# Key: the within transformation entails the computation of the difference between yit (actual value of y of i at time t) and the overtime mean at the individual level for each numeric column. 

# !!! we keep the reference to raw_data to avoid picking up the new columns just created from the between transformation
numeric_cols = raw_data.select_dtypes(include="number").columns

# !!! no need to groupby for raw_data[c] as it already cross matches with the mean of referenec given the index 
for c in numeric_cols: 
    trans_data[f"{c}_within"] = raw_data[c] - raw_data.groupby("country")[c].transform("mean")

trans_data

,year,country,i,P,W,WR,GDP,LS,PCOM,UN,...,WR_within,GDP_within,LS_within,PCOM_within,UN_within,SHORTUN_within,LONGUN_within,LF_within,REER_within,SH_within
0,1970-01-01,Australia,7.108333,9.1,6.413648,76.120158,2.835193e+05,59.7,2.113115,1.653364,...,16.910491,-3.776445e+05,1.638,-45.336529,-4.596866,NaN,NaN,-3538.42466,-1.197865,-0.630783
1,1971-01-01,Australia,7.008333,9.6,6.909308,77.071751,2.946057e+05,60.4,2.709369,1.907799,...,17.862084,-3.665581e+05,2.338,-44.740274,-4.342431,NaN,NaN,-3393.67766,1.927743,-0.730783
2,1972-01-01,Australia,5.083333,10.2,7.284607,75.808223,3.023225e+05,60.3,2.925285,2.610752,...,16.598556,-3.588412e+05,2.238,-44.524359,-3.639478,NaN,NaN,-3266.39166,3.140684,-2.655783
3,1973-01-01,Australia,6.983333,11.1,8.096658,74.465333,3.147389e+05,60.8,4.174010,2.309362,...,15.255666,-3.464249e+05,2.738,-43.275634,-3.940869,NaN,NaN,-3116.75066,14.883284,-0.755783
4,1974-01-01,Australia,13.158333,12.9,9.688749,75.817692,3.189447e+05,66.9,13.802962,2.670259,...,16.608025,-3.422190e+05,8.838,-33.646681,-3.579972,NaN,NaN,-2963.86566,15.983610,5.419217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2015-01-01,USA,0.226667,100.0,65.600000,62.655205,1.820602e+07,56.8,66.099106,5.279920,...,13.714370,6.489088e+06,-2.882,18.649462,-0.922572,-317.1236,672.3,27832.23500,10.296044,-6.130979
746,2016-01-01,USA,0.644167,101.3,66.400000,62.819300,1.850960e+07,56.8,56.023884,4.869110,...,13.878465,6.792669e+06,-2.882,8.574240,-1.333382,-539.4566,349.3,29889.53500,13.748199,-4.525864
747,2017-01-01,USA,1.152500,103.4,68.600000,63.695450,1.892457e+07,56.9,69.357989,4.355201,...,14.754615,7.207639e+06,-2.782,21.908346,-1.847291,-991.2066,32.3,31022.13500,13.405089,-4.031867
748,2018-01-01,USA,2.188333,105.9,70.900000,64.279238,1.948198e+07,56.8,89.358138,3.895676,...,15.338404,7.765040e+06,-2.882,41.908495,-2.306816,-1320.5396,-306.7,32777.33500,12.166410,-3.047915


In [64]:
# Joint variance (tabular)

In [65]:
# Joint variance (graphic)

##### <span style="color: dodgerblue"> **1.5) Plot the distribution of the one-way-fixed-effects-within x(it)-x(i.) and between (x(i.)) transformed dependent variable and of you key (preferred) explanatory variable (not all the explanatory variable) plotting on the same graph an histogram, a normal law with same empirical mean and standard error and a kernel continuous approximation. Comment the between and within difference for each variable, and compare within/within for dependent and explanatory variable, and between/between for dependent and explanatory variable: kurtosis, skewness, non-normality, high leverage observation (far from the mean), several modes (mixture of distribution)?** </span>

##### <span style="color: dodgerblue"> **1.6) FD. FIRST DIFFERENCE VARIABLE TRANSFORMATIONS: Compute the first-differences x(i,t)-x(i,t-1) for panel data. Check for the first 3 changes of individuals (for data sorted by individual and then time) in say 3T+1 first observations that when there is a change of individual in the stacked vector individuals x time, the first differences is a dot for “not available”. In other words, for first-differences for panel data, check that when you change individual, the first observation is missing with a dot, and it is not the difference of the first observation of the second individual minus the last observation of the first individual, for example.** </span>

##### <span style="color: dodgerblue"> **1.7) First differences distributions. Plot the distributions (histogram, KDE, normal law with same mean and standard error) for first difference dependent GDPG and first difference explanatory EDA/GDP for each of these two transformations.** </span>

##### <span style="color: dodgerblue"> **1.8) First differences simple correlation. For these FD transformed variables, plot the bivariate cloud of points with regression line and on top the marginal distribution of the horizontal axis and on right hand side the marginal distribution of the variable on the vertical axis. Compare with one-way-fixed effects and between distributions. Report the simple correlation coefficient on the graph.** </span>

##### <span style="color: dodgerblue"> **1.9) Restricted sample with a BALANCED PANEL: Two way fixed effects (TWFE) formula. Restrict the sample to the countries/individuals available with the longest duration (N=… countries over T=… periods). Compute -x(.t)+x(..), report the 6 numbers in a table as a function of time and plot them as a function of time, then comment. Then compute two-way-fixed-effects x(it)-x(i.)-x(.t)+x(..) transformed variables.** </span>

##### <span style="color: dodgerblue"> **1.10) TWFE Balanced panel. Compute descriptive statistics. Plot boxplots by country ordered by their variance from the smallest to the largest.** </span>

##### <span style="color: dodgerblue"> **1.11) TWFE Balanced panel. Present a table ordering the simple correlation coefficients of TWFE transformed GDPG and EDA/GDP by country from the largest positive to the lowest negative, with the standard error of GDPG and EDA in another column and the coefficient of simple regression: correlation coefficient * standard error of GDPG / standard error of EDA/GDP. Comment.** </span>

##### <span style="color: dodgerblue"> **1.12) UNBALANCED PANEL and TWFE transformation (remove countries with a single observation). Regress within transformed GDPG on time dummies and collect the residuals: this is the TWFE transformation. Regress within transformed EDA/GDP on time dummies and collect the residuals: this is the TWFE transformation. Alternatively, code the Wansbeek Kapstein (1989) transformation for two way fixed effects resulting in their equation 2.13 which is an extension of x(it)-x(i.)-x(.t)+x(..) obtained in the balanced panel case.** </span>

##### <span style="color: dodgerblue"> **1.13) TWFE unbalanced panel: plot the distribution (Kernel DE, histogram, corresponding normal law with the same two first moments). for dependent GDPG and explanatory EDA/GDP. Compare with one-way-fixed effects, between distributions.** </span>

##### <span style="color: dodgerblue"> **1.14) Plot boxplots of between distribution (all countries), then one-way and two-way-fixed effects and first differences distribution BY countries (or 20 individuals if your data set has more than 20 individuals), for the dependent variable and the key explanatory variables. Comment that you find the same insights from question 5. Comment on their differences of standard errors and means for each individuals** </span>

##### <span style="color: dodgerblue"> **1.15) Compute univariate descriptive statistics (min, Q1, median, Q3, max, mean, standard error) for one-way-Within, Between, two-way-fixed-effects and first differences transformed variables. Is the mean different from the median and why? How many standard errors from the mean are the MIN and MAX extremes. Report in the tables standardized MAX and MIN: (MAX-average)/standard error and (MIN-average)/standard error instead of MAX and MIN?** </span>

##### <span style="color: dodgerblue"> **1.16) Compare and comment the between versus one-way-within transformed bivariate correlation matrix for all variables (include a time trend 1,2,.,T) and with their lag (for time varying variables). Check poor simple correlation with the dependent variables and high correlation between explanatory variables.** </span>

##### <span style="color: dodgerblue"> **1.17) Comment the bivariate auto-correlation and trend-correlations (check the number of observations).** </span>

##### <span style="color: dodgerblue"> **1.18) In what follows, you do not need to include a deterministic time trend 1,2,.,T because the two transformations used eliminate it. Compare and comment of the two-way-within transformed bivariate simple correlation matrix of all the variables and another bivariate simple correlation matrix with all the first differences transformed variables (in the case of first differences, include also the lag of all variables). Check poor simple correlation below 0.1 with the dependent variables and high correlation between explanatory variables (over 0.8). Show the first 30 observations for the first differences and the lag of first differences. Check that each time you change individual, you have a dot for missing observation.** </span>

##### <span style="color: dodgerblue"> **1.19) Comment the bivariate graphs with linear, quadratic and Lowess fit for dependent and key explanatory variable (growth of gdp/head on vertical axis and aid/gdp): Within transformed, Between transformed, First differences, two-way-within transformed.** </span>

### **2) PART 2 - CLASSIC BENCHMARK MULTIVARIATE PANEL DATA ESTIMATORS**

##### <span style="color: orange"> **2.20) In a single table, report and comment the results of estimations of Between, Within (one-way fixed effects, (fe)) and Mundlak (random effects (re) including all X(i.) as regressors), two-way fixed effects (add year dummies in fe regression) and First differences, including all explanatory variables except the ones with high near-multicollinearity after their transformation.** </span>

##### <span style="color: orange"> **2.21) If, for the first differences dependent variable, it remains a simple auto-correlation above 0.1, a dynamic panel estimator can be tried. The estimators of the generalized method of moments (GMM) for panel data are only valid for short time panel T<10 and they face the issue of too many weak instruments. We suggest using its precursor, the Anderson-Hsiao (1981) estimator which allows to check the first stage of instrumental variables and to test for weak lagged instruments. Estimate an auto-regressive distributed lag (ARDL) model for dynamic panel data including the first lag of the dependent variable (for example: GDP per head growth) and the first lag of the key explanatory variable (for example: foreign aid/GDP), adding the first lag of other control variables is optional: Δ GDPGi,t = βy Δ GDPGi,t-1 + β1 Δ (aid/GDP)i,t + β2 Δ (aid/GDP)i,t-1 + Δ Controls i,t + Δ αi + Δ αt + Δ εi** </span>

!!! it has a lot of parts (check the paper)

### **2) PART 2B - OPTIONAL (if one of your variable is time-invariant z(i))**

##### <span style="color: orange"> **2.22) If one of your variable is time-invariant z(i), run a baseline Hausman Taylor estimation (pre-coded only in STATA) including all X(i.) as instruments. Comment the results. Else skip this question.** </span>

##### <span style="color: orange"> **2.23) If one of your variable is time-invariant z(i), run a between regression on z(i) explained by X(i.) and other time invariant variable (only with N observations). If the R2 is low, this may signal X(i.) are weak instruments poorly correlated with the variable z(i) to be instrumented. Comment. Else skip this question.** </span>

##### <span style="color: orange"> **2.24) If one of your variable is time-invariant z(i), as seen above, time invariant explanatory variables cannot explain the time varying within variance of the dependent variable and the Hausman Taylor internal instruments estimator is not so practical. Therefore, a practical shortcut is to include a time invariant variable multiplied by a time varying variable (interaction term): z(i) multiplied for x(it). Generate such a variable Include this product AND foreign aid into a one way fixed effects regression. Plot the estimated marginal effect (derivative) with respect to ICRG as a function of EDA/GDP (which is positive and goes as far as 20%).** </span>

### **3) PART 3 - OPEN SECTION**

##### <span style="color: red"> **3.25) Do whatever seem interesting to you in terms of original estimations (not already done by the replication of the original authors) with this database, present the table(s) in this file with comments, not only in the html output with code and output.** </span>